In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.data import AUTOTUNE
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Conv2D,Input,Flatten,MaxPool2D,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import MaxPooling2D

# Read CSV

In [ ]:
df = pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
df.head()

# Create Dataset

In [ ]:
X = []
Y = []
for i in df.values:
    X.append(i[1:].reshape((28,28,1)))
    Y.append(i[0])

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X_train,X_next, Y_train, Y_next = train_test_split(X,Y,random_state=46,test_size = 0.4,stratify=Y)
X_val, X_test, Y_val, Y_test = train_test_split(X_next,Y_next,random_state=46,test_size = 0.3,stratify=Y_next)

In [ ]:
Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)
Y_test = to_categorical(Y_test)

In [ ]:
def create_dataset(dataset):
    ds=tf.data.Dataset.from_tensor_slices(dataset)
    ds=ds.batch(32).prefetch(buffer_size=AUTOTUNE)
    return ds

trainds = create_dataset((X_train,Y_train))
testds = create_dataset((X_test,Y_test))
valds = create_dataset((X_val,Y_val))

# Create Model CNN

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3,3), padding='same', input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

# Train Model

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(trainds,epochs=10,validation_data=valds)

In [ ]:
model.evaluate(testds)

# Test Model To test.csv

In [ ]:
df1 = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
testX = []
for i in df1.values:
    testX.append(i[:].reshape((28,28,1)))

In [ ]:
testX = np.array(testX)

In [ ]:
pred = model.predict(testX)

In [ ]:
result_pred = [np.argmax(x) for x in pred]

In [ ]:
my_submission = pd.DataFrame({'ImageId': df1.index + 1, 'Label': result_pred})
my_submission.head()

In [ ]:
my_submission.to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(testX[3])